In [1]:
# import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
import operator
import statistics

# random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [2]:
# df_train = pd.read_csv("ecthr_train.csv")
# df_test = pd.read_csv("ecthr_test.csv")
df_train = pd.read_csv("bias_in_bios_train.csv")
df_test = pd.read_csv("bias_in_bios_test.csv")
# df_train = pd.read_csv("jigsaw_train.csv")
# df_test = pd.read_csv("jigsaw_test.csv")
# df = pd.concat([df1,df2,df3])
df_test

,Unnamed: 0,text,flipped_bio,swapped,gender,job_cat_id,M1,M2,Fairflow
0,31289,"He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",1,1,"She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac..."
1,61990,"He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",1,1,"She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc..."
2,29313,"His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",1,1,"Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Sh..."
3,60333,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,1,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...
4,15180,He was associated with such acts as Lil Wayne ...,She was associated with such acts as Lil Tammy...,She was associated with such acts as Lil Wayne...,1,1,She was associated with such acts as Lil Tammy...,She was associated with such acts as Lil Wayne...,She was associated with such acts as Lil Wayne...
...,...,...,...,...,...,...,...,...,...
3595,30273,Barbara works with a dedicated and terrific gr...,James works with a dedicated and terrific grou...,_ works with a dedicated and terrific group to...,0,0,James works with a dedicated and terrific grou...,James works with a dedicated and terrific grou...,David works with a dedicated and terrific grou...
3596,37506,She began her acting career as a lead in the D...,He began his acting career as a lead in the Do...,He began his acting career as a lead in the Do...,0,0,He began his acting career as a lead in the Do...,He began his acting career as a lead in the Do...,He began his acting career as a lead in the Do...
3597,35069,"Her parents are Heinz, a lawyer, and Gudrun Sc...","His parents are Heinz, a lawyer, and Gudrun Sc...","His parents are Heinz, a lawyer, and Gudrun Sc...",0,0,"His parents are Heinz, a lawyer, and Gudrun Sc...","His parents are Heinz, a lawyer, and Gudrun Sc...","His parents are Heinz, a lawyer, and Gudrun Sc..."
3598,11250,She is passionate about the welfare of the eld...,He is passionate about the welfare of the elde...,He is passionate about the welfare of the elde...,0,0,He is passionate about the welfare of the elde...,He is passionate about the welfare of the elde...,He is passionate about the welfare of the elde...


In [3]:
# from sklearn.utils import shuffle
# df = shuffle(df, random_state=42)
# # df = df[:100]

########################### Load dataset ###########################

In [4]:
# #load dataframe
# df

In [5]:
gender_counts_train = df_train['gender'].value_counts()
gender_counts_test = df_test['gender'].value_counts()

# Print the counts
print("train")
print(gender_counts_train)
print("test")
print(gender_counts_test)

train
gender
1    9477
0    8422
Name: count, dtype: int64
test
gender
1    1800
0    1800
Name: count, dtype: int64


Normal data

In [6]:
# Get the lists of sentences and their labels.
ap_type = "text"
X_train = list(df_train[ap_type].values)#text column here
y_train = list(df_train.job_cat_id.values)  #label column here
z_train = list(df_train.gender.values)  #label column here#label column here
X_val = df_test.text.values #text column here
y_val = df_test.job_cat_id.values #label column here
z_val = df_test.gender.values #label column here

# # Split dataset into train, validation, test

from sklearn.model_selection import train_test_split

X_train, X_val0, y_train, y_val0, z_train, z_val0 = train_test_split(X_train, y_train, z_train, test_size=0.00001, random_state=42)
# print(y_train)

Data with counterfactual data augmentation

In [7]:
# # Get the lists of sentences and their labels.
# ap_type = "flipped_text"
# X_train = list(df_train.text.values)+list(df_train[ap_type].values)#text column here
# y_train = list(df_train.defendant_state.values)+list(df_train.defendant_state.values)  #label column here
# z_train = list(df_train.gender.values)+list(df_train.gender.values)  #label column here#label column here
# X_val = df_test.text.values #text column here
# y_val = df_test.defendant_state.values #label column here
# z_val = df_test.gender.values #label column here

# # # Split dataset into train, validation, test

# from sklearn.model_selection import train_test_split

# X_train, X_val0, y_train, y_val0, z_train, z_val0 = train_test_split(X_train, y_train, z_train, test_size=0.00001, random_state=42)
# # print(y_train)

########################### Start training with checkpoint ###########################

In [8]:
# params
seq_len = 150
batch_size = batch_size = 32
# batch_size = batch_size = 8
epochs = 4

In [9]:
# !pip install transformers
# !pip install boto3
!pip install sentencepiece

Choose 1 type of model

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification, AlbertTokenizer, AlbertForSequenceClassification

from sent_modeling import BertForSequenceClassification as BertSentForSequenceClassification
from tokenization import BertTokenizer as SentTokenizer
from sent_modeling_albert import AlbertForSequenceClassification as AlbertSentForSequenceClassification 


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
)


# tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# model = AlbertForSequenceClassification.from_pretrained(
#     "albert-base-v2", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 2, # The number of output labels--2 for binary classification.
# )


# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
def data_prep(input, labels, gender, val_input, val_labels, val_gender):
        # tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        # seq_len = seq_len
        input_ids = tokenizer.batch_encode_plus(input, max_length = seq_len, truncation=True, return_tensors="pt", padding=True).input_ids
        labels_ids = torch.tensor(labels)
        gender_ids = torch.tensor(gender)
        val_input_ids = tokenizer.batch_encode_plus(val_input, max_length = seq_len, truncation=True, return_tensors="pt", padding=True).input_ids
        val_input_ids = tokenizer.batch_encode_plus(val_input, max_length = seq_len, truncation=True, return_tensors="pt", padding=True).input_ids
        val_labels_ids = torch.tensor(val_labels)
        val_gender_ids = torch.tensor(val_gender)

        train_dataset = TensorDataset(input_ids, labels_ids, gender_ids)
        val_dataset = TensorDataset(val_input_ids, val_labels_ids, val_gender_ids)
        print('{:>5,} training samples'.format(len(train_dataset)))
        print('{:>5,} validation samples'.format(len(val_dataset)))
        return train_dataset, val_dataset

In [12]:
from torch.utils.data import TensorDataset
train_dataset, val_dataset = data_prep(X_train, y_train, z_train, X_val, y_val, z_val)

17,898 training samples
3,600 validation samples


In [13]:


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# import numpy as np


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import random
from tqdm.notebook import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def train(model, optimizer, losslogger, start_epoch, epochs, run_id, train_dataloader, validation_dataloader, checkpoint_name):
    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss,
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    t = tqdm(range(start_epoch,epochs))

    for epoch_i in t:

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.classifier.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels
#             print(batch[0].shape)
#             print(batch[1].shape)
#             print(batch[2].shape)
            b_input_ids = batch[0].to(device)
            # b_input_mask = batch[2].to(device)
            b_labels = batch[1].to(device)


            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because
            # accumulating the gradients is "convenient while training RNNs".
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            outputs = model(b_input_ids,
                                 token_type_ids=None,
                                 # attention_mask=b_input_mask,
                                 labels=b_labels,
#                                  training_type="fine_tuning"
                                )

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_train_loss += outputs.loss.item()

            # Perform a backward pass to calculate the gradients.
            outputs.loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        # set in logs
        df = pd.DataFrame()
        df['chackpoint_name'] = pd.Series(checkpoint_name)
        df['epoch'] = pd.Series(epoch_i)
        df['Loss'] = pd.Series(outputs.loss.data.item())
        df['run'] = run_id
        # losslogger = losslogger.append(df)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
        # state = {'epoch': epoch_i + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'losslogger': losslogger, }
        # torch.save(state, f'{checkpoint_name}')

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels

            b_input_ids = batch[0].to(device)
            # b_input_mask = batch[2].to(device)
            b_labels = batch[1].to(device)


            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here:
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                outputs = model(b_input_ids,
                                       token_type_ids=None,
                                       # attention_mask=b_input_mask,
                                       labels=b_labels,
#                                        training_type="fine_tuning"
                                      )

            # Accumulate the validation loss.
            total_eval_loss += outputs.loss.item()

            # Move logits and labels to CPU
            logits = outputs.logits.detach().cpu().numpy()
            # label_ids = b_labels.to('cpu').numpy()
            y_label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, y_label_ids)


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )
    return avg_val_accuracy

#     print("")
#     print("Training complete!")

#     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

def load_checkpoint(model, optimizer, losslogger, filename):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, losslogger






In [14]:
seq_len = 150
vocab_length = 30522

import time
import datetime

from transformers.optimization import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = batch_size

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers.optimization import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = epochs

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)



/opt/conda/lib/python3.11/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# first training epoch
# Start
start_epoch = 0

# Logger
losslogger = pd.DataFrame()

# Checkpoint name
checkpoint_name = 'checkpoint.pth.tar'
train(model, optimizer, losslogger, start_epoch, epochs, 0, train_dataloader, validation_dataloader, checkpoint_name)

  0%|          | 0/4 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.



======== Epoch 1 / 4 ========
Training...
  Batch    40  of    560.    Elapsed: 0:00:08.
  Batch    80  of    560.    Elapsed: 0:00:16.
  Batch   120  of    560.    Elapsed: 0:00:23.
  Batch   160  of    560.    Elapsed: 0:00:31.
  Batch   200  of    560.    Elapsed: 0:00:38.
  Batch   240  of    560.    Elapsed: 0:00:46.
  Batch   280  of    560.    Elapsed: 0:00:54.
  Batch   320  of    560.    Elapsed: 0:01:01.
  Batch   360  of    560.    Elapsed: 0:01:09.
  Batch   400  of    560.    Elapsed: 0:01:17.
  Batch   440  of    560.    Elapsed: 0:01:24.
  Batch   480  of    560.    Elapsed: 0:01:32.
  Batch   520  of    560.    Elapsed: 0:01:40.

  Average training loss: 0.15
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.91
  Validation Loss: 0.26
  Validation took: 0:00:07

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    560.    Elapsed: 0:00:08.
  Batch    80  of    560.    Elapsed: 0:00:15.
  Batch   120  of    560.    Elapsed: 0:00:23.
  Batch  

0.9153761061946902

In [16]:


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# import numpy as np
def softmax(x):
    max = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
    e_x = np.exp(x - max) #subtracts each row with its max value
    sum = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
    f_x = e_x / sum
    return f_x

import numpy as np
from sklearn.metrics import f1_score

def flat_f1(preds, labels, average):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, average=average)


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Function to calculate the accuracy of our predictions vs labels
def confidence(preds, labels):
    # print(preds)
    pred_flat = softmax(preds)[:,0].flatten()
    # print("fd",pred_flat)
    labels_flat = labels.flatten()
    return np.sum(abs(pred_flat - labels_flat)) / len(labels_flat)

def perf_measure(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    y_actual, y_hat = labels_flat, pred_flat
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
    TPR = TP/(TP+FN+0.0000000000000000000000000000000000000000000001)
    TNR = TN/(TN+FP+0.0000000000000000000000000000000000000000000001)
    # print(TPR, TNR)
    return(TPR, TNR)

import random
from tqdm.notebook import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def test_eval(model, validation_dataloader):
    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss,
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # # For each epoch...
    # t = tqdm(range(start_epoch,epochs))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_f1 = 0
    total_eval_f1_micro = 0
    total_eval_f1_macro = 0
    total_eval_confidence = 0
    total_eval_TPR = 0
    total_eval_TNR = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels

        b_input_ids = batch[0].to(device)
        # b_input_mask = batch[1].to(device)
        b_labels = batch[1].to(device)


        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            # with torch.no_grad():
            outputs = model(b_input_ids,
                             token_type_ids=None,
                             # attention_mask=b_input_mask,
                             labels=b_labels,
                             # bertflow_path = bertflow_path
#                                  training_type="fine_tuning"
                            )
#                 # b_input_ids1 = torch.softmax(b_input_ids1, 2)
# #             # print(b_input_ids1.shape)
# #             outputs = model(b_input_ids1,
# #                                  # token_type_ids=None,
# #                                  attention_mask=b_input_mask,
# #                                  labels=b_labels,
# # #                                  training_type="fine_tuning"
# #                                 )
#                 outputs = model(b_input_ids1,
#                                        # token_type_ids=None,
#                                        attention_mask=b_input_mask,
#                                        labels=b_labels,
# #                                        training_type="fine_tuning"
#                                       )

        # Accumulate the validation loss.
        total_eval_loss += outputs.loss.item()

        # Move logits and labels to CPU
        logits = outputs.logits.detach().cpu().numpy()
        # label_ids = b_labels.to('cpu').numpy()
        y_label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, y_label_ids)
        total_eval_f1 += flat_f1(logits, y_label_ids, "binary")
        total_eval_f1_micro += flat_f1(logits, y_label_ids, "micro")
        total_eval_f1_macro += flat_f1(logits, y_label_ids, "macro")
        total_eval_confidence += confidence(logits, y_label_ids)
        TPR, TNR = perf_measure(logits, y_label_ids)
        total_eval_TPR += TPR
        total_eval_TNR += TNR
        # print(TPR, TNR, total_eval_TPR, len(validation_dataloader))


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy:",(avg_val_accuracy))

     # Report the final confidence for this validation run.
    avg_val_confidence = total_eval_confidence / len(validation_dataloader)
    print("  confidence: ", avg_val_confidence)

    # Report the final f1 for this validation run.
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    print("  f1: ", avg_val_f1)

     # Report the final f1 for this validation run.
    avg_val_f1_micro = total_eval_f1_micro / len(validation_dataloader)
    print("  f1_micro: ", avg_val_f1_micro)

     # Report the final f1 for this validation run.
    avg_val_f1_macro = total_eval_f1_macro / len(validation_dataloader)
    print("  f1_macro: ", avg_val_f1_macro)

     # Report the final TPR for this validation run.
    avg_val_TPR = total_eval_TPR / len(validation_dataloader)
    print("  TPR: ", avg_val_TPR)

    # Report the final TNR for this validation run.
    avg_val_TNR = total_eval_TNR / len(validation_dataloader)
    print("  TNR: ", avg_val_TNR)

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    print(avg_val_f1, avg_val_f1_micro, avg_val_f1_macro, avg_val_TPR, avg_val_TNR, avg_val_accuracy, avg_val_confidence)

    # # Record all statistics from this epoch.
    # training_stats.append(
    #     {
    #         'epoch': epoch_i + 1,
    #         'Training Loss': avg_train_loss,
    #         'Valid. Loss': avg_val_loss,
    #         'Valid. Accur.': avg_val_accuracy,
    #         'Training Time': training_time,
    #         'Validation Time': validation_time
    #     }
    # )
    return avg_val_accuracy

#     print("")
#     print("Training complete!")

#     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

def load_checkpoint(model, optimizer, losslogger, filename):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, losslogger






Input debias

In [17]:
import debias_class 

debiaser = debias_class.DebiasBert(tokenizer, None, 1,1)
old_emb = model.get_input_embeddings()
emb = debiaser.debias_full_word_embeddings(old_emb)
emb.weight.data = emb.weight.data.to(device)
model.set_input_embeddings(emb)

test
0.061112267855471976


In [18]:
import numpy as np
# first training epoch
# Start
start_epoch = 0

# Logger
losslogger = pd.DataFrame()

# Checkpoint name
checkpoint_name = 'checkpoint.pth.tar'

test_eval(model, validation_dataloader)
time.sleep(8)

Running Validation...


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metr

  Accuracy: 0.9056969026548672
  confidence:  0.9042374836297442
  f1:  0.4854253737804075
  f1_micro:  0.9056969026548672
  f1_macro:  0.6152122650660016
  TPR:  0.47013274336283184
  TNR:  0.44432153392330387
  Validation Loss: 0.47
  Validation took: 0:00:07
0.4854253737804075 0.9056969026548672 0.6152122650660016 0.47013274336283184 0.44432153392330387 0.9056969026548672 0.9042374836297442


In [19]:
import time
import datetime

from scipy.special import softmax

import itertools

seq_len = 100
vocab_length = 30522




# for model_type in models:
# for ind_key, dat in enumerate(data):
TPRD_scores = []
FPRD_scores = []
CF_scores = []
F_accuracy_scores = []
M_accuracy_scores = []
F_F1 = []
M_F1 = []

all_TPRD_scores = []
all_FPRD_scores = []
all_F_accuracy_scores = []
all_M_accuracy_scores = []
all_F_F1 = []
all_M_F1 = []

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# # from transformers.optimization import AdamW
# # # Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# # optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
# #                   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
# #                   eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
# #                 )



# # from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# # The DataLoader needs to know our batch size for training, so we specify it
# # here. For fine-tuning BERT on a specific task, the authors recommend a batch
# # size of 16 or 32.
# batch_size = 32

# from scipy.special import softmax
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# # For validation the order doesn't matter, so we'll just read them sequentially.
# validation_dataloader = DataLoader(
#             test_dataset, # The validation samples.
#             sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )

# ========================================
#               Testing
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.

print("")
print("Running test...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_test_prob = []
actual_label_list = []
gen_label_list = []
pred_label_list = []
# Evaluate data for one epoch
for batch in validation_dataloader:

    # Unpack this training batch from our dataloader.
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids
    #   [1]: attention masks
    #   [2]: labels

    b_input_ids = batch[0].to(device)
    # b_input_mask = batch[2].to(device)
    b_labels = batch[1].to(device)
    gen_labels = batch[2]


    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():
        outputs = model(b_input_ids,
                               token_type_ids=None,
                               # attention_mask=b_input_mask,
                               labels=b_labels,
#                                        training_type="fine_tuning"
                              )

    # Accumulate the validation loss.
    total_eval_loss += outputs.loss.item()

    # Move logits and labels to CPU
    logits = outputs.logits.detach().cpu().numpy()
    probs = softmax(logits, axis=1)
    pred_flat = list(np.argmax(logits, axis=1).flatten())
    label_ids = list(b_labels.to('cpu').numpy())
    label_ids2 = b_labels.to('cpu').numpy()
    gen_ids = list(gen_labels.to('cpu').numpy())

    for ind, lab in enumerate(label_ids):
#         print(probs[ind][lab])
        y_test_prob.append(probs[ind][lab])


    actual_label_list = (actual_label_list + label_ids)
    pred_label_list = (pred_label_list + pred_flat)
    gen_label_list = (gen_label_list + gen_ids)


#     # Calculate the accuracy for this batch of test sentences, and
#     # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids2)


print(len(actual_label_list))
print(len(gen_label_list))
print(len(pred_label_list))

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# # Calculate the average loss over all of the batches.
# avg_val_loss = total_eval_loss / len(validation_dataloader)

# # Measure how long the validation run took.
# validation_time = format_time(time.time() - t0)


# # Record all statistics from this epoch.
# training_stats.append(
#     {
#         'epoch': epoch_i + 1,
#         'Training Loss': avg_train_loss,
#         'Valid. Loss': avg_val_loss,
#         'Valid. Accur.': avg_val_accuracy,
#         'Training Time': training_time,
#         'Validation Time': validation_time
#     }
# )


#
import importlib
import evaluation
importlib.reload(evaluation)

# evaluation.reload(eval)
evaluate = evaluation.eval(2, pred_label_list, actual_label_list, gen_label_list)
# F_Y, M_Y, F_Y_predY, M_Y_predY = evaluate.TPR_nums(2)
# F_Y

#



import collections

class_list = [1]
class_list

# Function to calculate the demographic parity
# def flat_accuracy(preds, labels, gen):
preds = pred_label_list
labels = actual_label_list
gender = gen_label_list

job_0 = []
job_1 = []



class_dict = {1:job_1}
# class_dict = {0:job_0, 1:job_1}
class_dict

for i in class_list:
    for ind, label in enumerate(labels):
        if label == i:
#             print(i,preds[ind])
            class_dict[i].append((labels[ind], preds[ind], gender[ind]))
# class_dict




# TPR_diff scores
TPR_diff_list = []
for i in class_dict:
    TPR_diff_list.append(abs(evaluate.TPR_diff(i)))
# print("average TPR_diff:", statistics.mean(TPR_diff_list), min(TPR_diff_list), max(TPR_diff_list), np.std(TPR_diff_list))
# print("average TPR_diff:", statistics.mean(TPR_diff_list))
# print("min TPR_diff:", min(TPR_diff_list))
# print("max TPR_diff:", max(TPR_diff_list))
TPRD_scores.append(TPR_diff_list)


# FPR_diff scores
FPR_diff_list = []
for i in class_dict:
    FPR_diff_list.append(abs(evaluate.FPR_diff(i)))
# print("average FPR_diff:", statistics.mean(FPR_diff_list))
# print("min FPR_diff:", min(FPR_diff_list))
# print("max FPR_diff:", max(FPR_diff_list))
# print("average FPR_diff:", statistics.mean(FPR_diff_list), min(FPR_diff_list), max(FPR_diff_list)), np.std(FPR_diff_list)
FPRD_scores.append(FPR_diff_list)



f_job_list = [0]
m_job_list =[1]

# accuracy, f1 scores
f_accuracy_list = []
m_accuracy_list = []
f_f1_list = []
m_f1_list = []
f_accuracy_f_jobs_list = []
f_accuracy_m_jobs_list = []
m_accuracy_f_jobs_list = []
m_accuracy_m_jobs_list = []
for i in class_dict:
    f_accuracy, m_accuracy = evaluate.accuracy(i)
    f_f1, m_f1 = evaluate.F1(i)
    f_accuracy_list.append((f_accuracy))
    m_accuracy_list.append((m_accuracy))
    f_f1_list.append(f_f1)
    m_f1_list.append(m_f1)
    # if i in f_job_list:
    #     f_f_accuracy, m_f_accuracy = evaluate.accuracy(i)
    #     f_accuracy_f_jobs_list.append((f_f_accuracy))
    #     m_accuracy_f_jobs_list.append((m_f_accuracy))
    # else:
    #     f_m_accuracy, m_m_accuracy = evaluate.accuracy(i)
    #     f_accuracy_m_jobs_list.append((f_m_accuracy))
    #     m_accuracy_m_jobs_list.append((m_m_accuracy))
# print(statistics.mean(f_accuracy_list), min(f_accuracy_list),  max(f_accuracy_list))
# print(statistics.mean(m_accuracy_list), min(m_accuracy_list),  max(m_accuracy_list))
F_accuracy_scores.append(statistics.mean(f_accuracy_list))
M_accuracy_scores.append(statistics.mean(m_accuracy_list))
F_F1.append(statistics.mean(f_f1_list))
M_F1.append(statistics.mean(m_f1_list))


#  #
#     import importlib
#     import evaluation
#     importlib.reload(evaluation)

# #     combine factual and counterfactual
#     all_actual_label_list = (actual_label_list + cda_actual_label_list)
#     all_pred_label_list = (pred_label_list + cda_pred_label_list)
#     all_gen_label_list = (gen_label_list + cda_gen_label_list)

#     # evaluation.reload(eval)
#     evaluate = evaluation.eval(2, all_pred_label_list, all_actual_label_list, all_gen_label_list)
#     # F_Y, M_Y, F_Y_predY, M_Y_predY = evaluate.TPR_nums(2)
#     # F_Y

#



import collections

class_list = [1]
class_list

# Function to calculate the demographic parity
# def flat_accuracy(preds, labels, gen):
preds = pred_label_list
labels = actual_label_list
gender = gen_label_list

job_0 = []
job_1 = []



class_dict = {1:job_1}
# class_dict = {0:job_0, 1:job_1}
class_dict

for i in class_list:
    for ind, label in enumerate(labels):
        if label == i:
#             print(i,preds[ind])
            class_dict[i].append((labels[ind], preds[ind], gender[ind]))
# class_dict




# TPR_diff scores
TPR_diff_list = []
for i in class_dict:
    TPR_diff_list.append(abs(evaluate.TPR_diff(i)))
print("average TPR_diff:", statistics.mean(TPR_diff_list), min(TPR_diff_list), max(TPR_diff_list), np.std(TPR_diff_list))
# print("average TPR_diff:", statistics.mean(TPR_diff_list))
# print("min TPR_diff:", min(TPR_diff_list))
# print("max TPR_diff:", max(TPR_diff_list))
all_TPRD_scores.append(statistics.mean(TPR_diff_list))


# FPR_diff scores
FPR_diff_list = []
for i in class_dict:
    FPR_diff_list.append(abs(evaluate.FPR_diff(i)))
# print("average FPR_diff:", statistics.mean(FPR_diff_list))
# print("min FPR_diff:", min(FPR_diff_list))
# print("max FPR_diff:", max(FPR_diff_list))
print("average FPR_diff:", statistics.mean(FPR_diff_list), min(FPR_diff_list), max(FPR_diff_list)), np.std(FPR_diff_list)
all_FPRD_scores.append(statistics.mean(FPR_diff_list))





Running test...
3600
3600
3600
  Accuracy: 0.91
average TPR_diff: 0.11333333333333329 0.11333333333333329 0.11333333333333329 0.0
average FPR_diff: 0.18666666666666668 0.18666666666666668 0.18666666666666668


NameError: name 'data_names' is not defined